Training GAT network with 3 GAT layers + fixed floor area

In [1]:
import mlflow

# Specify tracking server
mlflow.set_tracking_uri(uri="http://localhost:5000")
mlflow.set_experiment("Predict bike traffic_GAT_fixed_floor_area")

<Experiment: artifact_location='mlflow-artifacts:/252164278876110764', creation_time=1724494965109, experiment_id='252164278876110764', last_update_time=1724494965109, lifecycle_stage='active', name='Predict bike traffic_GAT_fixed_floor_area', tags={}>

In [3]:
import os

os.environ['USE_PYGEOS'] = '0'
import datetime
from enum import Enum
import geopandas as gpd
import momepy
import networkx as nx
import numpy as np
from pathlib import Path
from shapely import LineString
import torch
# from torch_geometric.transforms import NormalizeScale
from torch_geometric.utils.convert import from_networkx
from utils.config import SIGNATURE


class OptimizerType(Enum):
    ADAM = {
        "learning_rate": 0.01,
        "betas": (0.9, 0.999),
        "eps": 1e-08,
        "weight_decay": 0.001,
    }
    SGD = {
        "learning_rate": 0.0001,
        "momentum": 0.8,
        "weight_decay": 0.0001,
    }


params = {
    "train_ratio": 0.8,
    "num_hidden_channels": 12,
    "GNN_dropout_rate": 0.1,
    "head_num": 9,
    #0:Adam, 1:SGD
    "optimizer": OptimizerType.ADAM,  # or OptimizerType.SGD  
    #  the learning rate of each parameter group using a cosine annealing schedule
    "scheduler": 0
    # "scheduler_gamma": [0.8],
    # "pos_weight": [1.3],
    # "model_embedding_size": [64],
    # "model_attention_heads": [3],
    # "model_layers": [4],
    # "model_dropout_rate": [0.2],
    # "model_top_k_ratio": [0.5],
    # "model_top_k_every_n": [1],
    # "model_dense_neurons": [256]
}

In [4]:
prefix_name = "testF4mean"
place_name = "Dresden"
file_name = place_name + "_" + prefix_name

len_normalize = 100

data_path = Path().resolve() / "data" / "processed" / f"{file_name}.gpkg"
traffic = gpd.read_file(data_path, driver='GPKG', layer='traffic')
print(f"loading data from {data_path}")

# traffic.crs = "epsg:4326"
traffic = traffic.to_crs(4839)
# traffic.crs = "epsg:3857"

loaded_graph = momepy.gdf_to_nx(traffic, approach="primal")
nodes = max(nx.connected_components(loaded_graph), key=len)
connected_graph = nx.subgraph(loaded_graph, nodes)
connected_graph = nx.Graph(connected_graph)
# H_nodes, H_edges = momepy.nx_to_gdf(H)
# print(H_edges.head(12))

num_idx_graph = nx.convert_node_labels_to_integers(
    connected_graph, label_attribute="coordinate")
node_loc = {
    i: num_idx_graph.nodes[i]["coordinate"]
    for i in range(num_idx_graph.number_of_nodes())
}

lineGraph = nx.line_graph(num_idx_graph)
# for node in lineGraph.copy():
#     lineGraph.add_node(node,mm_len=num_idx_graph.edges[node]["mm_len"]/len_normalize,
#                 occurrence=num_idx_graph.edges[node]["occurrence"],
#                 bikeability=num_idx_graph.edges[node]["bikeability"]
#                 )

attr_list = [
    "mm_len", "occurrence", "bikeability", "floor_area", "department_store",
    "supermarket", "kiosk", "variety_store", "foodstore", "bakery",
    "ice_cream", "bookstore", "ticket", "copyshop", "fashion", "DIY",
    "houseware", "furniture", "electronics", "sportstore", "florist",
    "laundry", "petstore", "toystore", "cafe", "restaurant", "pub", "theatre",
    "cinema", "market", "place_of_worship", "bank", "pharmacy", "chemist",
    "post_office", "townhall", "library", "kindergarten", "school", "college",
    "park", "stadium", "sportplace", "university"
]
attr_list_no_occurrence = [
    "mm_len", "bikeability", "floor_area", "department_store", "supermarket",
    "kiosk", "variety_store", "foodstore", "bakery", "ice_cream", "bookstore",
    "ticket", "copyshop", "fashion", "DIY", "houseware", "furniture",
    "electronics", "sportstore", "florist", "laundry", "petstore", "toystore",
    "cafe", "restaurant", "pub", "theatre", "cinema", "market",
    "place_of_worship", "bank", "pharmacy", "chemist", "post_office",
    "townhall", "library", "kindergarten", "school", "college", "park",
    "stadium", "sportplace", "university"
]
for node in lineGraph.copy():
    attr_dict = {
        attr_name: num_idx_graph.edges[node][attr_name]
        for attr_name in attr_list
    }
    lineGraph.add_node(node, **attr_dict)

data = from_networkx(lineGraph, group_node_attrs=attr_list_no_occurrence)
data.x[:, 0] = data.x[:, 0] / len_normalize
# data.x = F.normalize(data.x, dim=0)
# data.x = z_score(data.x)
data.y = data.occurrence.unsqueeze(1)
del data.occurrence
# data = transform(data)
# type(pyg_graph)
print(data)
# print(nx.number_of_edges(lineGraph))
# L2.edges
# print(pyg_graph.num_nodes)


loading data from E:\MyDoc\Deep_Thinkvitality\MLcode\Bike_Prediction\data\processed\Dresden_testF4mean.gpkg
Data(edge_index=[2, 55050], x=[12912, 43], y=[12912, 1])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

_, gdf_edges = momepy.nx_to_gdf(connected_graph)
fig, ax = plt.subplots(1, 1, figsize=(6,6))
gdf_edges.plot(
        column="occurrence",
        ax=ax,
        legend=True, 
        legend_kwds={"shrink": 0.5},
        norm=colors.LogNorm(vmin=1, 
        vmax=gdf_edges["occurrence"].max())
)
ax.set_axis_off()

fig_path = Path().resolve()/"data"/"image"/f"{file_name}.svg"
plt.savefig(fig_path, format="svg")


In [ ]:
# find histogram of features
# idx = 0
# test = data.x[:,idx]
# plt.hist(test,bins=20,range=(0, 5))
# print(attr_list_no_occurrence[idx])
# print(max(test))


In [ ]:
# gdf_edges.explore()

Split the data to trains and test sets

In [5]:
now = datetime.datetime.now()
time_tag = now.strftime("%m%d_%H%M")

data.train_mask = torch.rand(data.num_nodes) < params["train_ratio"]
data.test_mask = ~data.train_mask
train_mask_path = Path().resolve(
) / "data" / "mask" / f"{file_name}_{time_tag}.npy"
np.save(train_mask_path, data.train_mask)

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
from torch.nn import Linear, MSELoss
import torch.nn.functional as F
# from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv, TopKPooling

# hidden_channels=16


# Define the GAT model
class GAT(torch.nn.Module):

    def __init__(self, hidden_channels):
        super(GAT, self).__init__()

        # GNN layers
        self.conv1 = GATConv(data.num_features,
                             hidden_channels,
                             heads=params["head_num"])
        self.conv2 = GATConv(hidden_channels * params["head_num"],
                             hidden_channels,
                             heads=params["head_num"])
        self.conv3 = GATConv(hidden_channels * params["head_num"],
                             hidden_channels,
                             heads=1,
                             concat=False)

        # linear layers
        self.linear1 = Linear(hidden_channels, 24)
        self.linear2 = Linear(24, 24)
        self.linear3 = Linear(24, 1)

    def forward(self, x, edge_index):
        # First Message Passing Layer (Transformation)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=params["GNN_dropout_rate"], training=self.training)

        # Second Message Passing Layer
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=params["GNN_dropout_rate"], training=self.training)

        # Third Message Passing Layer
        x = self.conv3(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=params["GNN_dropout_rate"], training=self.training)

        # linear layer
        x = self.linear1(x)
        x = F.elu(x)
        x = self.linear2(x)
        x = F.elu(x)
        x = self.linear3(x)
        return x


model = GAT(params["num_hidden_channels"])
print(model)
print(f"Number of parameters: {count_parameters(model)}")


GAT(
  (conv1): GATConv(43, 12, heads=9)
  (conv2): GATConv(108, 12, heads=9)
  (conv3): GATConv(108, 12, heads=1)
  (linear1): Linear(in_features=12, out_features=24, bias=True)
  (linear2): Linear(in_features=24, out_features=24, bias=True)
  (linear3): Linear(in_features=24, out_features=1, bias=True)
)
Number of parameters: 19225


In [8]:
from torch_geometric.loader import ClusterData, ClusterLoader

torch.manual_seed(12345)
cluster_data = ClusterData(data, num_parts=512)  # 1. Create subgraphs. 128
train_loader = ClusterLoader(
    cluster_data, batch_size=128,
    shuffle=True)  # 2. Stochastic partioning scheme. 32

print()
total_num_nodes = 0
for step, sub_data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of nodes in the current batch: {sub_data.num_nodes}')
    print(sub_data)
    print()
    total_num_nodes += sub_data.num_nodes

print(f'Iterated over {total_num_nodes} of {data.num_nodes} nodes!')

Computing METIS partitioning...



Step 1:
Number of nodes in the current batch: 3222
Data(x=[3222, 43], y=[3222, 1], train_mask=[3222], test_mask=[3222], edge_index=[2, 9940])

Step 2:
Number of nodes in the current batch: 3230
Data(x=[3230, 43], y=[3230, 1], train_mask=[3230], test_mask=[3230], edge_index=[2, 10042])

Step 3:
Number of nodes in the current batch: 3225
Data(x=[3225, 43], y=[3225, 1], train_mask=[3225], test_mask=[3225], edge_index=[2, 9480])

Step 4:
Number of nodes in the current batch: 3235
Data(x=[3235, 43], y=[3235, 1], train_mask=[3235], test_mask=[3235], edge_index=[2, 9418])

Iterated over 12912 of 12912 nodes!


Done!


In [11]:
# Initialize model
# mlflow.pytorch.autolog()
# model = GCN(hidden_channels)

# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
data = data.to(device)

# Initialize Optimizer
optimizer_type = params["optimizer"]
optimizer_config = optimizer_type.value

if params["optimizer"] == OptimizerType.ADAM:
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=optimizer_config["learning_rate"],
                                 betas=optimizer_config["betas"],
                                 eps=optimizer_config["eps"],
                                 weight_decay=optimizer_config["weight_decay"])
    print(f'Setting up Adam optimizer with '
          f'lr: {optimizer_config["learning_rate"]}, '
          f'betas: {optimizer_config["betas"]}, '
          f'eps: {optimizer_config["eps"]}, '
          f'weight_decay: {optimizer_config["weight_decay"]}')

elif params["optimizer"] == OptimizerType.SGD:
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=optimizer_config["learning_rate"],
                                momentum=optimizer_config["momentum"],
                                weight_decay=optimizer_config["weight_decay"])
    print(f'Setting up SGD optimizer with '
          f'lr: {optimizer_config["learning_rate"]}, '
          f'momentum: {optimizer_config["momentum"]}, '
          f'weight_decay: {optimizer_config["weight_decay"]}')

if params["scheduler"] == 1:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                           T_max=500,
                                                           eta_min=0.000001)

# Define loss function
criterion = MSELoss()
# criterion = L1Loss()


def train():
    model.train()
    running_loss = 0.0
    step = 0

    for sub_data in train_loader:  # Iterate over each mini-batch.
        optimizer.zero_grad()
        # Use all data as input, because all nodes have node features
        # out = model(data.x, data.edge_index)
        out = model(sub_data.x,
                    sub_data.edge_index)  # Perform a single forward pass.

        # Only use nodes with labels available for loss calculation --> mask
        # loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss = criterion(
            out[sub_data.train_mask], sub_data.y[sub_data.train_mask]
        )  # Compute the loss solely based on the training nodes.
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        step += 1

    return running_loss / step


def test():
    model.eval()
    out = model(data.x, data.edge_index)
    # rmse = RMSE(data.y[data.test_mask], out[data.test_mask])
    loss = criterion(out[data.test_mask], data.y[data.test_mask])
    return loss


# def RMSE(v, v_):
#     """
#     Mean squared error.
#     :param v: torch array, ground truth.
#     :param v_: torch array, prediction.
#     :return: torch scalar, RMSE averages on all elements of input.
#     """
#     return torch.sqrt(torch.mean((v_ - v) ** 2))
# return torch.mean((v_ - v) ** 2)

# animation setting
do_animation = 0
animation_count = 0
max_groud_value = max(data.y.detach().numpy())
mask = data.train_mask.numpy()

max_epoch = 5001
max_early_stopping_counter = 500
# best_train_loss = 100000
best_test_loss = 100000
nocheck_until = 3000
early_stopping_counter = 0

# train_losses = []
# test_losses = []

with mlflow.start_run() as run:

    mlflow.log_param("dataset", file_name)
    mlflow.log_param("num_params", count_parameters(model))
    # mlflow.log_artifact(fig_path)
    mlflow.log_artifact(train_mask_path)

    # Log the general parameters
    for key in params:
        if key != "optimizer":  # Exclude logging the optimizer enum object directly
            mlflow.log_param(key, params[key])

    # Log the optimizer type
    mlflow.log_param("optimizer_type", optimizer_type.name)

    # Log each parameter within the optimizer config
    for key, value in optimizer_config.items():
        mlflow.log_param(f"{optimizer_type.name.lower()}_{key}", value)

    for epoch in range(max_epoch):
        if early_stopping_counter <= max_early_stopping_counter:
            #train
            train_loss = np.sqrt(train())
            mlflow.log_metric(key="Train loss",
                              value=float(train_loss),
                              step=epoch)

            #test
            test_loss = torch.sqrt(test())
            mlflow.log_metric(key="Test loss",
                              value=float(test_loss),
                              step=epoch)

            # if epoch > nocheck_until: #do not check early_stopping after some epochs
            #       if float(test_loss) < best_test_loss:
            #             best_loss = test_loss
            #             early_stopping_counter = 0
            #             # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)

            #       if float(train_loss) < best_train_loss:
            #             best_loss = train_loss
            #             early_stopping_counter = 0
            #             mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
            #       else:
            #             early_stopping_counter += 1

            # if epoch > nocheck_until: #do not check early_stopping after some epochs
            #       if float(test_loss) < best_loss:
            #             best_loss = test_loss
            #             early_stopping_counter = 0
            #             mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
            #       else:
            #             early_stopping_counter += 1

            if epoch > nocheck_until:  #do not check early_stopping after some epochs

                if float(test_loss) < best_test_loss:
                    best_test_loss = test_loss
                    mlflow.pytorch.log_model(model,
                                             "model",
                                             signature=SIGNATURE)
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
            # scheduler.step()
        else:
            print("Early stopping due to no improvement.")
            break

        if epoch % 200 == 0:
            print(
                f"Epoch: {epoch:03d}, Train Loss: {train_loss:.1f}, Test Loss: {test_loss:.1f}"
            )

        if epoch % 10 == 0 and do_animation == 1:
            prediction = model(data.x, data.edge_index)
            truth = data.y.detach().numpy()
            prediction = prediction.detach().numpy()
            pair_predict = np.concatenate((truth, prediction), axis=1)

            animation_path = Path().resolve()/"data"/"anima"/\
                  f"{file_name}_{time_tag}_{animation_count:03d}.png"

            scatter1 = plt.plot(pair_predict[mask.transpose().ravel(), 0],
                                pair_predict[mask.transpose().ravel(), 1],
                                'o',
                                color='royalblue',
                                markersize=2,
                                label='train data')
            scateer2 = plt.plot(pair_predict[~mask.transpose().ravel(), 0],
                                pair_predict[~mask.transpose().ravel(), 1],
                                'o',
                                color='indianred',
                                markersize=2,
                                label='test data')

            plt.text(-300, -300, f'epoch: {epoch:04d}', fontsize='medium')
            plt.xlabel("Ground Truth")
            plt.ylabel("Prediction")
            plt.xlim([0, max_groud_value])
            plt.ylim([0, max_groud_value])
            plt.legend(loc='upper left')
            plt.savefig(animation_path, format="png", dpi=200)
            plt.close()

            animation_count += 1

    # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)

    # plot

    result_path = Path().resolve(
    ) / "data" / "image" / f"{file_name}_{time_tag}.svg"
    result_Emap_path = Path().resolve(
    ) / "data" / "image" / f"{file_name}_{time_tag}_Emap.svg"

    prediction = model(data.x, data.edge_index)
    truth = data.y.detach().numpy()
    prediction = prediction.detach().numpy()
    error = prediction - truth
    pair_predict = np.concatenate((truth, prediction), axis=1)

    plt.plot(pair_predict[:, 0], pair_predict[:, 1], "o", markersize=3)
    plt.plot(pair_predict[data.test_mask, 0],
             pair_predict[data.test_mask, 1],
             "or",
             markersize=3)
    plt.xlabel("Ground Truth")
    plt.ylabel("Prediction")
    plt.xlim([0, max(pair_predict[:, 0])])
    plt.ylim([0, max(pair_predict[:, 0])])
    plt.savefig(result_path, format="svg")
    plt.show()

    lines = [
        LineString([node_loc[node[0]], node_loc[node[1]]])
        for node in lineGraph.nodes
    ]
    readout_gdf = gpd.GeoDataFrame({
        "error": error.ravel(),
        "geometry": lines
    },
                                   crs="EPSG:4839")

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    readout_gdf.plot(
        column="error",
        ax=ax,
        legend=True,
        legend_kwds={"shrink": 0.4},
        # norm=colors.LogNorm(vmin=1, vmax=1200)
    )
    ax.set_axis_off()
    plt.savefig(result_Emap_path, format="svg")
    plt.show()

    mlflow.log_artifact(result_path)
    mlflow.log_artifact(result_Emap_path)

Setting up Adam optimizer with lr: 0.01, betas: (0.9, 0.999), eps: 1e-08, weight_decay: 0.001
Epoch: 000, Train Loss: 166.4, Test Loss: 143.9


2024/08/25 11:38:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-stork-436 at: http://localhost:5000/#/experiments/252164278876110764/runs/30167ff1d2e64493a43aaf78126245b1.
2024/08/25 11:38:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/252164278876110764.


KeyboardInterrupt: 

In [ ]:
n_bins = 50
bin_range1 = (0, np.max(truth))
bin_range2 = (0, np.max(prediction))
# bin_range2 = (0, 200)


fig, axes = plt.subplots(1,2, figsize=(6,3), sharey=True)
axes[0].hist(truth, bins=n_bins, range=bin_range1)
axes[0].set_title('Ground truth')
axes[0].set_xlim(left=0)

axes[1].hist(prediction, bins=n_bins, range=bin_range2)
axes[1].set_title('Prediction')
axes[1].set_xlim(left=0)

# plt.xlim(bin_range)
plt.tight_layout()
plt.show()

In [ ]:
# Define number of groups
num_groups = 5
max_value = 1000

pair_predict = np.concatenate((truth, prediction), axis=1)

# Split data into groups based on values of the first element
grouped_data = [pair_predict[(pair_predict[:, 0] >= i * max_value / num_groups) & 
                             (pair_predict[:, 0] < (i + 1) * max_value / num_groups)] for i in range(num_groups)]

# Plot histograms for each group
fig, axs = plt.subplots(num_groups, 1, figsize=(6, 12), sharex=True)
for i in range(num_groups):
    axs[i].hist(grouped_data[i][:, 1], bins=50)
    axs[i].set_title(f'Group {i+1}: {i*1000/num_groups} - {(i+1)*1000/num_groups}')

plt.show()


In [ ]:
plt.plot(pair_predict[:,0], pair_predict[:,1], "o", markersize=3)
plt.plot(pair_predict[data.test_mask,0], pair_predict[data.test_mask,1], "or", markersize=3)
plt.xlabel("Ground Truth")
plt.ylabel("Prediction")
plt.xlim([0, 200])
plt.ylim([0, 200])
plt.show()

In [ ]:
n_bins = 50
bin_range = (-400, 400)
plt.hist(error, bins=n_bins, range=bin_range)
plt.title('Prediction Error')
plt.xlim(bin_range)
plt.show()